In [20]:
# coding: utf-8
"""
igraphを使ったグラフクラスタリング
greedyアルゴリズム（louvain法）によってクラスタリングした後
サブグラフの中心性指標を図ることで単語の順位付けを行う
personalized_pagerankによるp(word|topic)の計算
pagerankを無向ではなく、有効で計算
purityによるスコアの計算
p(word|topic)から凸２次計画問題を解くことで、p(topic)を求める
サンプル文の抽出
"""
from igraph import *
import csv
import collections
import pickle
import numpy as np
from openopt import QP

# csvファイルの読み込み
def readcsv(path):
    f = open(path, "rb")
    dataReader = csv.reader(f)
    arr = [row for row in dataReader]
    return arr

def writecsv(arr, path):
    f = open(path, "ab")
    dataWriter = csv.writer(f)
    dataWriter.writerows(arr)
    f.close()

def readdump(path):
    f = open(path, "r")
    arr = pickle.load(f)
    f.close()
    return arr

# 有向エッジリストを入力して、重み付き無向ネットワークを出力する
def cal_edgelist_to_network(list_edge):
    # 有向エッジリストを無向エッジリストに変換する
    non_list_edge = [tuple(sorted(row)) for row in list_edge]
    list_edge = [tuple(row) for row in list_edge]
    # ノードリスト
    list_vertices = list(set([word for row in list_edge for word in row]))
    # エッジリストとそのweightを作成
    non_tuple_edge, non_tuple_weight = zip(*collections.Counter(non_list_edge).items())
    tuple_edge, tuple_weight = zip(*collections.Counter(list_edge).items())
    return {"vertex": list_vertices, "edge": list(non_tuple_edge), "weight": list(non_tuple_weight), "directed_edge": list(tuple_edge), "directed_weight": list(tuple_weight)}

# クラスタリング済みのネットワークを元にサブグラフのリストを作成
# vertexには全てのvertexを代入する（PageRankを計算するため）
def cal_cluster_to_network(dict_network):
    if dict_network.has_key("cluster") == False:
        print "クラスタリングができていません"
    
    # クラスタごとにwordをまとめる
    dict_cluster = collections.defaultdict(list)
    for word, cluster in zip(dict_network["vertex"], dict_network["cluster"]):
        dict_cluster[cluster].append(word)

    # リストに変換
    list_cluster_vertex = [row[1] for row in dict_cluster.items()]
    
    # 同様にエッジとウェイトのリストも作成する
    list_cluster_edge = []
    list_cluster_weight = []
    for cluster_vertex in list_cluster_vertex:
        list_cluster_edge_one = []
        list_cluster_weight_one = []
        # エッジリストの中に、一つでもノードが含まれていれば、そのクラスのノードに含める
        for row, weight in zip(dict_network["directed_edge"], dict_network["directed_weight"]):
            # and と or を切り替えることによって性能の比較
            if row[0] in cluster_vertex or row[1] in cluster_vertex:
                list_cluster_edge_one.append(row)
                list_cluster_weight_one.append(weight)
        list_cluster_edge.append(list_cluster_edge_one)
        list_cluster_weight.append(list_cluster_weight_one)
    
    # まとめる
    list_dict_network = [{"vertex": dict_network["vertex"],
                          "edge": cluster_edge,
                          "weight": cluster_weight}
                         for cluster_edge, cluster_weight
                         in zip(list_cluster_edge, list_cluster_weight)]
    
    return list_dict_network

# f_measureを計算する
def cal_f_measure(list_predict_measure):
    # 生成したクラスタ内のカウント
    dict_predict_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_predict_cluster[row[0]].append(row[1])
        
    # もとあるクラス内のカウント
    dict_measure_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_measure_cluster[row[1]].append(row[0])
    
    # local_purityの計算
    list_purity = []
    for row in dict_predict_cluster.items():
        major_class = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        class_num = len(row[1])
        list_purity.append([major_class, class_num])
    purity = float(np.sum(zip(*list_purity)[0])) / np.sum(zip(*list_purity)[1])
    print "Purity: ", purity
    
    # inverse_purityの計算
    list_inverse_purity = []
    for row in dict_measure_cluster.items():
        major_class = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        class_num = len(row[1])
        list_inverse_purity.append([major_class, class_num])
    inverse_purity = float(np.sum(zip(*list_inverse_purity)[0])) / np.sum(zip(*list_inverse_purity)[1])
    print "Inverse Purity: ", inverse_purity
    
    print "F-value: ", 2 / (1 / purity + 1 / inverse_purity)
    
# 凸２次計画問題を解いてp(topic)を求めるための関数
def cal_prob_topic(dict_network_master, list_dict_network_sub):
    prob_master = np.array([row[1] for row in sorted(zip(dict_network_master["vertex"], dict_network_master["page_rank"]), key=lambda x: x[0])])
    
    for i, dict_network_sub in enumerate(list_dict_network_sub):
        if i == 0:
            prob_sub = np.array([row[1] for row in sorted(zip(dict_network_sub["vertex"], dict_network_sub["page_rank"]), key=lambda x: x[0])])
        else:
            list_tmp = np.array([row[1] for row in sorted(zip(dict_network_sub["vertex"], dict_network_sub["page_rank"]), key=lambda x: x[0])])
            prob_sub = np.vstack((prob_sub, list_tmp))
    
    H = 2 * prob_sub.dot(prob_sub.T)
    f = -2 * prob_master.dot(prob_sub.T)
    Aeq = np.ones(len(list_dict_network_sub))
    beq = 1
    lb = np.zeros(len(list_dict_network_sub))
    
    p = QP(H, f, Aeq=Aeq, beq=beq, lb=lb)
    r = p.solve("cvxopt_qp")
    k_opt = r.xf
    return k_opt

###メイン部分
1. 有向エッジリストから無向重み付きエッジリストを作成
2. louvain法によるクラスタリング
3. クラスタごとにpagerankを計算し、p(word|topic)とする
4. クラスタごとにp(word|topic)を出力する辞書を作成

In [23]:
# エッジリストの読み込み
list_edge = readcsv("./files/kaigo_honne/list_edgelist20160127.csv")
# 元のネットワークを作成する（無向）
dict_network_master = cal_edgelist_to_network(list_edge)
# g = Graph(directed=True)
g_master = Graph()
g_master.add_vertices(dict_network_master["vertex"])
g_master.add_edges(dict_network_master["edge"])
# louvain法によるクラスタリング、vertexと同じ長さのクラスタ番号が書かれたリストがreturn
dict_network_master["cluster"] = g_master.community_fastgreedy(weights=dict_network_master["weight"]).as_clustering().membership
# 元のネットワークのpagerankを求める
g_master_rev = Graph()
g_master_rev.add_vertices(dict_network_master["vertex"])
g_master_rev.add_edges(dict_network_master["directed_edge"])
dict_network_master["page_rank"] = g_master.personalized_pagerank(directed=True, weights=dict_network_master["directed_weight"])
# クラスタ結果をもとにサブグラフのリストを作成
list_dict_network_sub = cal_cluster_to_network(dict_network_master)
# サブクラスタごとに中心性の計算_
for i, dict_network_sub in enumerate(list_dict_network_sub):
    g_sub = Graph()
    g_sub.add_vertices(dict_network_sub["vertex"])
    g_sub.add_edges(dict_network_sub["edge"])
    list_dict_network_sub[i]["center_bet"] = g_sub.betweenness(directed=False, weights=dict_network_sub["weight"])
    list_dict_network_sub[i]["center_eigen"] = g_sub.eigenvector_centrality(directed=False, weights=dict_network_sub["weight"])
    list_dict_network_sub[i]["page_rank"] = g_sub.personalized_pagerank(directed=True, weights=dict_network_sub["weight"])
print "クラスタ数: ", len(list_dict_network_sub)

# トピックごとにwordを入力したらp(word|topic)が出るような辞書を作成
list_dict_prob = []
for i in range(len(list_dict_network_sub)):
    list_word_page = sorted(zip(list_dict_network_sub[i]["vertex"], list_dict_network_sub[i]["page_rank"]), key=lambda x: x[1], reverse=True)
    list_dict_prob.append({row[0]: row[1] for row in list_word_page})
    
# 凸２次計画問題を解いて、p(topic)を求める
list_prob_topic = cal_prob_topic(dict_network_master, list_dict_network_sub)

クラスタ数:  7

------------------------- OpenOpt 0.5625 -------------------------
problem: unnamed   type: QP
solver: cvxopt_qp
     pcost       dcost       gap    pres   dres
 0: -1.6080e-03 -1.0022e+00  1e+00  3e-17  3e+00
 1: -1.6088e-03 -1.2176e-02  1e-02  1e-16  3e-02
 2: -1.6572e-03 -2.1835e-03  5e-04  8e-17  2e-03
 3: -1.7434e-03 -1.7945e-03  5e-05  7e-17  1e-18
 4: -1.7510e-03 -1.7550e-03  4e-06  1e-16  1e-18
 5: -1.7511e-03 -1.7512e-03  6e-08  5e-17  1e-18
 6: -1.7511e-03 -1.7511e-03  6e-10  1e-16  2e-18
Optimal solution found.
istop: 1000 (optimal)
Solver:   Time Elapsed = 0.0 	CPU Time Elapsed = 0.0
objFuncValue: -0.0017511414 (feasible, MaxResidual = 0)


In [30]:
num = 1
for row in sorted(list_dict_prob[num].items(), key=lambda x: x[1], reverse=True):
    print row[0], row[1]

利用 0.0207933068933
場所 0.0199554629773
駅 0.0176797758104
やすい 0.0167686742249
いい 0.0162893902437
便利 0.0152539116885
可能 0.0149842882185
大変 0.0146918912226
車 0.0144734825271
バス 0.0140940113621
環境 0.0134067968987
訪問 0.0123563585195
気 0.0100063131133
静か 0.00988481801687
近い 0.0095105503712
不便 0.00886327789043
アクセス 0.00769858699914
ホーム 0.00760172188148
緑 0.00746816979671
スーパー 0.0073499400918
自然 0.00708743506469
周辺 0.00673776726364
徒歩 0.00664279826473
無い 0.00642902552799
散歩 0.00635875957554
車椅子 0.00614366017375
母 0.00607282043707
外 0.00589935112135
高齢者 0.00586119015329
自宅 0.00571775683559
距離 0.00558027697015
周り 0.00532514806343
コンビニ 0.005298003571
買い物 0.00506646671974
面会 0.00503341177712
公園 0.00493115229775
立地 0.00481398177867
移動 0.00455207549392
遠い 0.00429926282261
難しい 0.00419482993822
電車 0.00408070526542
基本的 0.00405319368948
道路 0.00400976939141
人 0.00391190309112
気軽 0.00380175398889
景色 0.0037765023026
館内 0.00376111957874
お見舞い 0.00371397062182
雰囲気 0.00371344573574
駐車場 0.00368052001158
病院 0.003

### 定量評価部分
1. 形態素解析済みセンテンスを読み込み、確率を計算し、どのクラスに属するか計算
2. 予測クラスと実際クラスのリストを作成する

In [28]:
list_sep_words = readdump("./files/kaigo_honne/list_sep_words_label.dump")
list_words= readcsv("./files/kaigo_honne/list_sentence_user_label.csv")
# 確率が最大になるクラスを予想する
# 実質ラベルがintじゃない場合は、エラーとして、記録しない
list_predict_measure = []
list_words_rev = []
error_count = 0
for row, row1 in zip(list_sep_words, list_words):
    try:
        class_tmp = 0
        prob_tmp = 0
        predict_class = []
        for num, dict_prob in enumerate(list_dict_prob):
            prob_tmp_tmp = 1
            for word in row[0]:
                prob_tmp_tmp *= float(dict_prob[word])
            prob_tmp_tmp *= list_prob_topic[num]
            if prob_tmp_tmp > prob_tmp:
                class_tmp = num
                prob_tmp = prob_tmp_tmp
        list_predict_measure.append([class_tmp, row[1]])
        list_words_rev.append(row1)
    except:
        error_count += 1
print "エラーデータ数: ", error_count

エラーデータ数:  68


###計算結果の表示

In [29]:
cal_f_measure(list_predict_measure)

Purity:  0.489826905557
Inverse Purity:  0.635590646827
F-value:  0.553269138333


In [18]:
cal_f_measure(list_predict_measure)

Purity:  0.492256301245
Inverse Purity:  0.639234740358
F-value:  0.556199417134
